In [2]:
from datetime import datetime
from pathlib import Path
root = Path().cwd()

import numpy as np
import xarray as xr

from matplotlib import pyplot as plt
import matplotlib.ticker as ticker
import cartopy.crs as ccrs
import cartopy.feature as cfeature

import coordinate_utils as cu
import vec_transform as vt
import trend_utils as tu

import pickle

In [5]:
with open(root/'clusters.pkl', 'rb') as f:
    clusters = pickle.load(f)

In [6]:
clusters

{'k': [{'center': {'msl': array([[101120.16 , 101129.375, 101127.67 , ..., 100825.32 , 100823.55 ,
            100821.62 ],
           [101111.35 , 101114.734, 101114.71 , ..., 100823.1  , 100821.35 ,
            100819.59 ],
           [101104.9  , 101106.164, 101105.89 , ..., 100820.81 , 100818.86 ,
            100817.14 ],
           ...,
           [101478.945, 101478.02 , 101476.87 , ..., 101082.164, 101092.22 ,
            101107.484],
           [101464.3  , 101463.36 , 101462.516, ..., 101069.055, 101080.7  ,
            101101.36 ],
           [101450.22 , 101449.36 , 101448.55 , ..., 101060.32 , 101072.97 ,
            101096.12 ]], dtype=float32),
    'z': array([[54604.61 , 54613.61 , 54626.605, ..., 54460.402, 54460.15 ,
            54459.973],
           [54620.145, 54630.797, 54643.33 , ..., 54477.97 , 54477.89 ,
            54477.82 ],
           [54644.555, 54653.207, 54662.31 , ..., 54495.62 , 54495.645,
            54495.664],
           ...,
           [57625.023, 5